In [1]:
import pandas as pd
import numpy as np
import os
import inspect
from datetime import datetime
from timeit import default_timer as timer
from matplotlib import pyplot as plt
import gc
from dataclasses import dataclass
from pathlib import Path
import pickle
from sklearn.cluster import KMeans
import sys
import threading
import time
import psutil
import scipy
import scipy.spatial

In [6]:
## Global Settings & Variables
# System path
cwd = Path.cwd()
src_path = cwd.parents[2]

# Column in the data frame that is used to determine the identity of a dataset
identifier_column = "FileName"

# Map part that is used in calculations
lon_min = -122.50
lon_max = -122.35
lat_min = 37.60
lat_max = 37.85

# Amount of grid cells used. Default amount (8 x 8) is based upon Dr. Maouche's descriptions and depictions
num_lon_bins = 8
num_lat_bins = 8

# Timer
start_timer = timer()
func_timer = 0

## Memory monitoring
# Global variable to hold memory usage
# Enable / disable memory output in console
memory_monitor_output = False
current_memory_usage = 0
memory_threshold = 85
memory_highest_value = 0
stop_event = threading.Event()


date_time_now = datetime.now()
date_time_now_str = date_time_now.strftime("%H:%M:%S")
print(f"Starting execution at: {date_time_now_str}")


Starting execution at: 12:15:36


In [7]:
## Load cabspotting sets from txt files
files_path = Path("Daten/cabspottingdata - Copy")
source_files_path =  os.path.join(src_path, files_path)

all_files = os.listdir(source_files_path)

text_files = [file for file in all_files if file.endswith('.txt')]

select_files = text_files 

# Global variable that contains all original cab datasets as a list of data frames
profiles_list = []

for file_name in select_files:
    file_path = os.path.join(source_files_path, file_name)
    df = pd.read_csv(file_path, sep=" ", header=None, names=["Latitude", "Longitude", "Occupied", "Timestamp"])
    df["FileName"] = file_name[:-4]
    df = df.drop(columns="Occupied")
    profiles_list.append(df)

In [8]:
## Load synthetic cabspotting sets from pickle

# Files path
files_path = Path("Repository/LDTP_Trace/notebooks-ldptrace/output/cabspotting-single/pickles")
source_files_path =  os.path.join(src_path, files_path)

all_files = os.listdir(source_files_path)

pickle_files = [file for file in all_files if file.endswith('.pkl')]

# Global variable that contains all synthetic cab datasets as a list of data frames
synthetic_profiles = []

for file_name in pickle_files:
    with open(f"{source_files_path}/{file_name}", 'rb') as f:
        trajectories = pickle.load(f)

    data_points = []

    # By default, synthetic trajectories contain multiple sub trajectories with one or multiple data points. To allow processing in HMC, this dimensionality is removed.
    # All subtrajectories are split and merged into single data points, conserving the original order.
    for trajectory in trajectories:
        for coord_pair in trajectory:
            longitude, latitude = coord_pair
            data_points.append([latitude, longitude])
            
    df = pd.DataFrame(data_points, columns=["Latitude", "Longitude"])
    df["FileName"] = file_name[:-4]
    synthetic_profiles.append(df)
    


In [9]:
def monitor_memory(stop_event):

    global current_memory_usage
    global memory_highest_value
    global memory_monitor_output
    
    while not stop_event.is_set():
        mem = psutil.virtual_memory()
        current_memory_usage = mem.percent

        if mem.percent > memory_highest_value:
            memory_highest_value = mem.percent

        if memory_monitor_output:
            print(f"Current memory usage: {mem.percent}, highest memory usage: {memory_highest_value}")
            
        time.sleep(1)  

In [10]:
def split_single_dataframe(selected_profile):

    """
    (DEPRECATED) Splits a single trajectory (data set) in sub trajectories, depending on the value in "Occupied". A sub trajectory beigns and ends when the value of "Occupied" changes
    """
    
    # Find start of trajectory
    start_traj = selected_profile['Occupied'].diff() == 1

    # Find end of trajcetory
    end_traj = selected_profile['Occupied'].diff() == -1

    # Create new trajectory
    selected_profile['Trajectory_ID'] = (start_traj.cumsum() + end_traj.cumsum()).fillna(0).astype(int)

    # Group data frame by trajectory id
    grouped = selected_profile.groupby('Trajectory_ID')

    # Save trajectories in dictionary
    trajectories_list = [group.copy() for _, group in grouped]

    return trajectories_list

In [11]:
def split_data_frame_list_by_date(profiles_list):

    """
    Splits a single data set into two (one as a baseline data set, the other one as a comparee data set) - Based upon date (Timestamp)
    """

    first_80_percent = []
    last_20_percent = []

    # 29.05.2008 -> 50/50 Split     06.04.2008 -> 80/20 Split
    time_limit_str = "29.05.2008 23:59"
    datetime_limit = datetime.strptime(time_limit_str,'%d.%m.%Y %H:%M')


    for df in profiles_list:
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
        df = df.sort_values('Timestamp')

        split_index = df[df['Timestamp'] > datetime_limit].first_valid_index()

        if split_index is not None:
            first_80_df = df.loc[:split_index - 1]
            first_80_df = first_80_df.loc[::5]
            last_20_df = df.loc[split_index:]
            last_20_df = last_20_df.loc[::5]
        else:
            first_80_df = df
            last_20_df = pd.DataFrame()

        first_80_percent.append(first_80_df)
        last_20_percent.append(last_20_df)

    return first_80_percent, last_20_percent

In [12]:
def split_data_frame_list_top_30(profiles_list):

    """
    Splits a single data set in half - Based upon the 30 most active days (days with most data points) for each individual profile
    """

    first_part = []
    last_part = []

    for df in profiles_list:

        df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
        df['Date'] = df['Timestamp'].dt.date
        
        date_grouped = df.groupby('Date').size().reset_index(name='counts')
        
        top_30_days = date_grouped.nlargest(30, 'counts')['Date']
        
        first_half_df = pd.DataFrame()
        last_half_df = pd.DataFrame()
        
        for day in top_30_days:
            day_entries = df[df['Date'] == day]
            day_entries = day_entries.sort_values('Timestamp')
            mid_point = len(day_entries) // 2
            
            first_half = day_entries.iloc[:mid_point]
            last_half = day_entries.iloc[mid_point:]
            
            first_half_df = pd.concat([first_half_df, pd.DataFrame(first_half)], ignore_index=True)
            last_half_df = pd.concat([last_half_df, pd.DataFrame(last_half)], ignore_index=True)
        
        first_part.append(first_half_df)
        last_part.append(last_half_df)

    return first_part, last_part


In [13]:
def split_data_frame_list_randomly(profiles_list, seed = 123):

    """
    Splits each DataFrame in a list of DataFrames into two halves with random lengths
    """
    np.random.seed(seed)

    first_part = []
    last_part = []

    for df in profiles_list:
        if len(df) > 1:
            split_index = np.random.randint(1, len(df))
            first_part_df = df.iloc[:split_index]
            last_part_df = df.iloc[split_index:]
        else:
            first_part_df = df
            last_part_df = pd.DataFrame()

        first_part.append(first_part_df)
        last_part.append(last_part_df)

    return first_part, last_part


In [14]:
def split_data_frame_list_by_half(profiles_list):

    """
    Splits each DataFrame in a list of DataFrames into two halves
    """

    first_50_percent = []
    last_50_percent = []

    for df in profiles_list:
        if len(df) > 1:
            split_index = len(df) // 2
            first_part_df = df.iloc[:split_index]
            last_part_df = df.iloc[split_index:]
        else:
            first_part_df = df
            last_part_df = pd.DataFrame()

        first_50_percent.append(first_part_df)
        last_50_percent.append(last_part_df)

    return first_50_percent, last_50_percent


In [15]:
def split_data_frame_list_by_cluster(profiles_list):

    first_half_list = []
    last_half_list = []

    for profile in profiles_list:
        if len(profile) > 1:  # Check if the DataFrame contains at least two data points
            # Calculate the number of clusters, limited to a minimum of 3 and a maximum of 5
            num_clusters = max(3, min(5, len(profile) // 2))
            # Perform K-Means clustering to identify clusters of geographic regions
            kmeans = KMeans(n_clusters=num_clusters, random_state=42)
            profile['Cluster'] = kmeans.fit_predict(profile[['Latitude', 'Longitude']])

            # Randomly split each cluster into two parts
            cluster_data_frames = []
            for cluster in range(num_clusters):
                cluster_data = profile[profile['Cluster'] == cluster]
                split_point = len(cluster_data) // 2
                shuffled_indices = np.random.permutation(cluster_data.index)
                first_part = cluster_data.loc[shuffled_indices[:split_point]]
                second_part = cluster_data.loc[shuffled_indices[split_point:]]
                cluster_data_frames.append((first_part, second_part))

            # Combine the parts of all clusters
            first_parts = pd.concat([df[0] for df in cluster_data_frames], ignore_index=True)
            last_parts = pd.concat([df[1] for df in cluster_data_frames], ignore_index=True)

            # Store the parts in the lists
            first_half_list.append(first_parts[['Longitude', 'Latitude', 'FileName']])
            last_half_list.append(last_parts[['Longitude', 'Latitude', 'FileName']])
        else:
            # If there are fewer than two data points, add the entire DataFrame to both lists
            first_half_list.append(profile[['Longitude', 'Latitude', 'FileName']])
            last_half_list.append(profile[['Longitude', 'Latitude', 'FileName']])

    return first_half_list, last_half_list


In [16]:
def calculate_map_bins(long_upper = lon_max, long_lower = lon_min, lat_upper = lat_max, lat_lower = lat_min, num_lon_bins = num_lon_bins, num_lat_bins = num_lat_bins):
    """
    Calculates the map bins of the grid, based upon the specified values for latitude, longitude and the amount of grid cells.
    
    Args:
    - long_upper: Upper limit of the longitude.
    - long_lower: Lower limit of the longitude.
    - lat_upper: Upper limit of the latitude.
    - lat_lower: Lower limit of the latitude.
    - num_lon_bins: Number of longitude bins.
    - num_lat_bins: Number of latitude bins.
    
    Returns:
    - bin_rows: DataFrame containing the calculated bins with their limits.
    """
    
    long_total = long_upper - long_lower
    lat_total = lat_upper - lat_lower

    long_bin_size = long_total / num_lon_bins
    lat_bin_size = lat_total / num_lat_bins

    bin_rows = pd.DataFrame({
        "Index": range(num_lat_bins * num_lon_bins)
    })

    bin_rows['Longitude Lower Limit'] = long_lower + (bin_rows['Index'] % num_lon_bins) * long_bin_size
    bin_rows['Longitude Upper Limit'] = bin_rows['Longitude Lower Limit'] + long_bin_size
    bin_rows['Latitude Upper Limit'] = lat_upper - (bin_rows['Index'] // num_lon_bins) * lat_bin_size
    bin_rows['Latitude Lower Limit'] = bin_rows['Latitude Upper Limit'] - lat_bin_size

    return bin_rows

map_bins = calculate_map_bins()

In [1]:
def calculate_distance_between_points(array_a, array_b, distance_type="hausdorff", seed=0):
    if distance_type == "topsoe":
        return calculate_topsoe_divergence(array_a, array_b)
    elif distance_type == "hausdorff":
        return calculate_hausdorff_distance(array_a, array_b, seed)
    else:
        raise Exception("Missing selection of distance calculation algorithm!")
    

In [18]:
def calculate_topsoe_divergence(P_ij, Q_ij):
    return np.sum(P_ij * np.log(2 * P_ij / (P_ij + Q_ij))) + np.sum(Q_ij * np.log(2 * Q_ij / (P_ij + Q_ij)))

In [35]:
def calculate_hausdorff_distance(array_a, array_b, seed):
    return scipy.spatial.distance.directed_hausdorff(array_a, array_b, seed)[0]

In [20]:
def create_heatmap(df, lon_bins = num_lon_bins, lat_bins = num_lat_bins, density = True):
    """
    Create a heat map based on the given DataFrame. By default returns a probability density heat map.
    
    Args:
    - df: DataFrame containing 'Latitude' and 'Longitude' columns.
    - lon_min: Minimum longitude for the bins.
    - lon_max: Maximum longitude for the bins.
    - lat_min: Minimum latitude for the bins.
    - lat_max: Maximum latitude for the bins.
    - num_lon_bins: Number of longitude bins.
    - num_lat_bins: Number of latitude bins.
    - density: If True, normalize the heat map to represent probability density.
    
    Returns:
    - hist: 2D histogram representing the heat map.
    """
    
    lat = df["Latitude"].values
    lon = df["Longitude"].values

    # Create grid from longitude / latitude data points
    lon_bins = np.linspace(lon_min, lon_max, num_lon_bins)
    lat_bins = np.linspace(lat_min, lat_max, num_lat_bins)

    # Create 2D histogram
    hist, _, _ = np.histogram2d(lon, lat, bins=[lon_bins, lat_bins], density=density)
    
    # Replace zero values with a small number to avoid issues with log calculations
    hist[hist == 0] = 1e-10
    
    return hist


In [21]:
def show_heat_map(heat_map, title = ""):

    """
    Displays given heat map in the console
    """

    plt.imshow(heat_map.T, origin='lower', extent=[lon_min, lon_max, lat_min, lat_max], aspect='auto', cmap='magma')

    plt.colorbar()
    plt.title(title)
    
    plt.show()

In [22]:
def visited_bins(profile, map_bins = map_bins):
    """
    Calculates the visited bins (grid cells) of a given DataFrame
    
    Args:
    - profile: DataFrame containing longitude and latitude columns.
    - map_bins: DataFrame containing the bin definitions with columns:
        ['Longitude Lower Limit', 'Longitude Upper Limit', 
         'Latitude Lower Limit', 'Latitude Upper Limit', 'Index']
         
    Returns:
    - sorted_bins: Array of unique visited bin indices, sorted.
    """
    
    # Initialize an empty list to store the indices of visited bins
    visited_bins = []

    # Iterate through each bin definition in map_bins
    for _, bin_row in map_bins.iterrows():
        # Find all profile entries that fall within the current bin
        in_bin = profile[
            (profile["Longitude"] >= bin_row["Longitude Lower Limit"]) &
            (profile["Longitude"] < bin_row["Longitude Upper Limit"]) &
            (profile["Latitude"] >= bin_row["Latitude Lower Limit"]) &
            (profile["Latitude"] < bin_row["Latitude Upper Limit"])
        ]
        if not in_bin.empty:
            visited_bins.append(bin_row["Index"])

    # Get unique and sorted bin indices
    unique_bins = np.unique(visited_bins)
    sorted_bins = np.sort(unique_bins)
    
    return sorted_bins


In [23]:
def remove_profiles_out_of_bounds_coordinates(profiles_list, lon_max = lon_max, lon_min = lon_min, lat_max = lat_max, lat_min = lat_min):
    """
    Removes coordinate points that are outliers and outside the specified part of the map 
    (outside min/max latitude/longitude range) to prevent out of bound errors.
    
    Args:
    - profiles_list: List of DataFrames containing profiles with 'Longitude' and 'Latitude' columns.
    - lon_max: Maximum allowed longitude.
    - lon_min: Minimum allowed longitude.
    - lat_max: Maximum allowed latitude.
    - lat_min: Minimum allowed latitude.
    
    Returns:
    - cleaned_profiles_list: List of DataFrames with out of bounds coordinates removed.
    """
    
    cleaned_profiles_list = []
    
    for df in profiles_list:
        cleaned_df = df[
            (df["Longitude"] <= lon_max) &
            (df["Longitude"] >= lon_min) &
            (df["Latitude"] <= lat_max) &
            (df["Latitude"] >= lat_min)
        ]
        cleaned_profiles_list.append(cleaned_df.reset_index(drop=True))
    
    return cleaned_profiles_list


In [24]:
def area_coverage_precision(visited_bins_h, visited_bins_candidate):
    """
    Formula to calculate utility (area coverage). See paper for reference.
    """
    if len(visited_bins_candidate) == 0:
        return 0
    return len(set(visited_bins_h).intersection(visited_bins_candidate)) / len(visited_bins_candidate)


In [25]:
def area_coverage_recall(visited_bins_h, visited_bins_candidate):
    """
    Formula to calculate utility (area coverage). See paper for reference.
    """
    if len(visited_bins_h) == 0:
        return 0
    return len(set(visited_bins_h).intersection(visited_bins_candidate)) / len(visited_bins_h)

In [26]:
def calculate_area_coverage(visited_bins_h, visited_bins_candidate):
    """
    Calculate utility (area coverage) between two heat maps. See paper for reference.
    """
    precision = area_coverage_precision(visited_bins_h, visited_bins_candidate)
    recall = area_coverage_recall(visited_bins_h, visited_bins_candidate)
    return (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

In [27]:
def update_alteration_counter(counter, h, h_dash, u, v, reset):

    """
    During heat map alteration process, reset counter if topsoe divergence between H & V gets smaller (faster) compared to topsoe divergence between H & F. There exists
    a specified diff_previous value to prevent extremly long loops with barely any improvements
    """
    
    if reset:
        counter = 0 
    else:
        counter += 1 
    return counter

In [28]:
def get_profile_u(heat_map_h, precomputed_profiles):

    """
    Selects profile u, the profile with the highest similarity (lowest topsoe divergence)
    """

    smallest_value = np.inf
    selected_candidate_id = None

    # Determine profile u, profile with the smallest topsoe divergence to the profile that shall be obfuscated, that is not the profile to obfuscate itself
    for profile_candidate_id, data_candidate in precomputed_profiles.items():

        topsoe_distance = calculate_distance_between_points(heat_map_h, data_candidate.heat_map)
        
        if topsoe_distance < smallest_value and topsoe_distance >= 0:
            smallest_value = topsoe_distance
            selected_candidate_id = profile_candidate_id
    
    return precomputed_profiles[selected_candidate_id]

In [29]:
def get_profile_v(profile_h, precomputed_profiles):
    """
    Select profile v, the profile with the highest area coverage (utility) with profile h (profile to obfuscate).

    Args:
    - profile_h: The profile to be obfuscated.
    - precomputed_profiles: A dictionary containing precomputed profiles.

    Returns:
    - The profile with the highest area coverage with profile_h.
    """
    biggest_value = 0
    selected_candidate_id = None
    profile_h_visited_bins = profile_h.visited_bins

    # Remove profile_h from precomputed_profiles
    precomputed_profiles_sans_h = {k: v for k, v in precomputed_profiles.items() if k != profile_h.profile_data.iloc[0][identifier_column]}

    # Determine profile v, profile with the best utility (highest area coverage)
    for profile_candidate_id, data_candidate in precomputed_profiles_sans_h.items():
        if len(data_candidate.visited_bins) != 0:
            f_score = calculate_area_coverage(profile_h_visited_bins, data_candidate.visited_bins)
        else:
            print(f"0 Bins found for profile {profile_candidate_id}?!")
            continue

        if f_score > biggest_value:
            biggest_value = f_score
            selected_candidate_id = profile_candidate_id
    
    return precomputed_profiles[selected_candidate_id]


In [30]:
def heat_map_alteration(profile_h, obfuscation_factor, all_precomputed_profiles, max_count_iterations):
    """
    Modify a heat map based on given datasets and obfuscation parameters.

    Args:
    - profile_h: The target profile to be obfuscated.
    - obfuscation_factor: The factor determining the extent of obfuscation.
    - all_precomputed_profiles: A dictionary containing precomputed profiles.
    - max_count_iterations: The maximum number of iterations allowed for alteration.

    Returns:
    - Modified heat map of profile_h.
    - Counter for the number of iterations performed.
    """
    gc.collect()

    try:
        func_timer = timer()
        print(f"Starting {inspect.currentframe().f_code.co_name} at: {datetime.now()}")

        # Get the number of data points in profile_h
        amount_data_points_H = len(profile_h.profile_data)
        # Get the identifier column from profile_h
        profile_h_id = profile_h.profile_data.iloc[0][identifier_column]
        # Get the heat map of profile_h
        heat_map_h = profile_h.heat_map

        counter_while = 0
        
        # Get profile_u based on heat_map_h
        timer_u = timer()
        profile_u = get_profile_u(heat_map_h, all_precomputed_profiles)
        
        profile_u_id = profile_u.profile_data.iloc[0][identifier_column]

        # If h and u belong to a different owner, no need to obfuscation
        if profile_u_id != profile_h_id:
            print(f"Profile U: {profile_u_id} Profile H: {profile_h_id} belong to a different owner. No need for obfuscation. Returning H.")
            return heat_map_h, counter_while

        # Filter out profile_u from all_precomputed_profiles
        filtered_precomputed_profiles = {profile_id: data for profile_id, data in all_precomputed_profiles.items() if profile_id != profile_u_id}

        # Get profile_u again after filtering
        profile_u = get_profile_u(heat_map_h, filtered_precomputed_profiles)

        profile_u_id = profile_u.profile_data.iloc[0][identifier_column]

        # Further filter to ensure profile_u is not used for profile_v selection
        filtered_precomputed_profiles_for_v = {profile_id: data for profile_id, data in filtered_precomputed_profiles.items() if profile_id != profile_u_id}

        # Get profile_v based on profile_h_visited_bins
        profile_v = get_profile_v(profile_h, filtered_precomputed_profiles_for_v)
        
        profile_v_id = profile_v.profile_data.iloc[0][identifier_column]

        counter_obfuscation = 0

        # Get heat maps of profile_u and profile_v
        heat_map_u = filtered_precomputed_profiles[profile_u_id].heat_map
        heat_map_v = all_precomputed_profiles[profile_v_id].heat_map

        # Debug output to ensure U and V are different
        #print(f"Profile H ID: {profile_h_id},Profile U ID: {profile_u_id}, Profile V ID: {profile_v_id}")
        #print(f"Topsoe Divergence U-V: {calculate_distance_between_points(heat_map_u, heat_map_v)}")

        if profile_u_id == profile_v_id:
            raise ValueError("Profile U and Profile V should not be the same")

        heat_map_h_dash = heat_map_h
        diff_previous = 0

        while (calculate_distance_between_points(heat_map_h_dash, heat_map_v) > calculate_distance_between_points(heat_map_h_dash, heat_map_u) 
                and counter_obfuscation <= max_count_iterations and counter_while < 150000): # Hard cap for iterations to avoid out of memory scenarios
            
            #mem = psutil.virtual_memory() 
            #if mem.percent > memory_threshold:
            #    print("Memory usage too high. Cancelling Heat Map Alteration and returning Heat Map V as a substitute.")
            #    return heat_map_v, counter_while

            # Calculate matrices for obfuscation
            matrix_r = heat_map_h_dash * amount_data_points_H
            matrix_w = heat_map_h_dash * heat_map_v * (1 - heat_map_u)
            matrix_o = matrix_r + ((obfuscation_factor/np.sum(matrix_w)) *  matrix_w)
            matrix_h_dash = (1 / amount_data_points_H) * matrix_o

            topsoe_divergence_h_v = calculate_distance_between_points(heat_map_h_dash, heat_map_v)
            topsoe_divergence_h_u = calculate_distance_between_points(heat_map_h_dash, heat_map_u)

            counter_while += 1

            if (topsoe_divergence_h_v - topsoe_divergence_h_u) < diff_previous:
                counter_obfuscation = update_alteration_counter(counter_obfuscation, heat_map_h_dash, matrix_h_dash, heat_map_u, heat_map_v, True)
                heat_map_h_dash = matrix_h_dash
                diff_previous = topsoe_divergence_h_v - topsoe_divergence_h_u
                continue

            counter_obfuscation = update_alteration_counter(counter_obfuscation, heat_map_h_dash, matrix_h_dash, heat_map_u, heat_map_v, False)
            heat_map_h_dash = matrix_h_dash
            diff_previous = topsoe_divergence_h_v - topsoe_divergence_h_u

        print(f"Iteration {counter_while} finished. Topsoe Divergence H V = {calculate_distance_between_points(heat_map_h_dash, heat_map_v)} Topsoe Divergence H U = {calculate_distance_between_points(heat_map_h_dash, heat_map_u)}")
        print(f"{inspect.currentframe().f_code.co_name} execution finished after {timer() - func_timer} seconds")

        if counter_obfuscation >= max_count_iterations:
            print(f"No solution found within {counter_while} iterations. Returning heat map V as a substitute.")
            return heat_map_v, counter_while

        return heat_map_h_dash, counter_while

    finally:
        # Cleanup code to free resources
        gc.collect()
        #print(f"Resources have been cleaned up after {inspect.currentframe().f_code.co_name}")


In [31]:
def modify_number_of_records(mobility_trace, amount_target_datapoints):
    P = mobility_trace.copy()
    P['Timestamp'] = pd.to_datetime(P['Timestamp'], format='%d.%m.%Y %H:%M', errors='coerce')
    P_dash = pd.DataFrame(columns=P.columns)

    while len(P_dash) < amount_target_datapoints:
        P_dash = pd.concat([P_dash, P.iloc[[0]]], ignore_index=True)

        for i in range(1, len(P)):
            p_mid = pd.DataFrame(index=[0], columns=P.columns)
            p_mid["Longitude"] = (P["Longitude"].iloc[i - 1] + P["Longitude"].iloc[i]) / 2
            p_mid["Latitude"] = (P["Latitude"].iloc[i - 1] + P["Latitude"].iloc[i]) / 2
            
            timestamp_difference = (P["Timestamp"].iloc[i] - P["Timestamp"].iloc[i - 1]) / 2
            p_mid["Timestamp"] = P["Timestamp"].iloc[i - 1] + timestamp_difference

            P_dash = pd.concat([P_dash, p_mid], ignore_index=True)
            P_dash = pd.concat([P_dash, P.iloc[[i]]], ignore_index=True)
            #print("Added entry at index", i)

        P = P_dash.copy()

    print(P.head(100))
    P["FileName"] = P["FileName"][0]
    #print(P)
    print(f"Original Length: {len(mobility_trace)} New Length: {len(P)}")
    #random_selection = random.choices(P, weights = None, cum_weights = None, k = amount_target_datapoints)
    #print(random_selection)
    return P

In [37]:
### Scenario 1: Execute AP-Attack on the HMC obfuscated data sets  

"""
Select which data set shall be used
!! Important: The synthetic_profiles by default DO NOT have a Timestamp component, which means that splits BY DATE are NOT possible !!
"""

# Allowed keywords "SYNTHETIC" or "NORMAL"
scenarios = ["SYNTHETIC", "NORMAL"]
# Allowed keywords "HALF" or "RANDOM" or "CLUSTER"
split_algorithm = ["RANDOM"]

# Seeds generated with https://www.random.org/ True Random Number Generator, Params Min: 1, Max: 10000
random_seeds = [8046]#, 8251, 8166, 435, 7782, 9807, 3582, 21, 5446, 781]



def process_data(split_algorithm, scenarios, seed):
    try:
        for algorithm in split_algorithm:
            for scenario in scenarios:
                dataset_type = scenario

                # Select data set
                if dataset_type == "SYNTHETIC":
                    profiles_list_cleansed = remove_profiles_out_of_bounds_coordinates(synthetic_profiles)
                    print("Synthetic data set selected")
                elif dataset_type == "NORMAL":
                    profiles_list_cleansed = remove_profiles_out_of_bounds_coordinates(profiles_list)
                    print("Normal data set selected")
                else:
                    print(f"No dataset {scenario} could be found!")
                    sys.exit()

                # Select split function
                if algorithm == "HALF":
                    first_50, last_50 = split_data_frame_list_by_half(profiles_list_cleansed)
                elif algorithm == "RANDOM":
                    first_50, last_50 = split_data_frame_list_randomly(profiles_list_cleansed, seed)
                elif algorithm == "CLUSTER":
                    first_50, last_50 = split_data_frame_list_by_cluster(profiles_list)
                else:
                    print(f"No Split Algorithm {algorithm} could be found!")
                    continue

                first_50 = list(filter(lambda df: not df.empty, first_50))
                last_50 = list(filter(lambda df: not df.empty, last_50))

                output_folder_path = Path('Daten/Ausgabe/AP Scenario HMC 50_50')
                output_folder = src_path / output_folder_path
                os.makedirs(output_folder, exist_ok=True)

                results = []

                save_interval = 5  # save every 5 results
                now = datetime.now()
                formatted_date = now.strftime("%Y%m%d_%H%M%S")
                output_path = os.path.join(output_folder, f"ap_output_HMC_{dataset_type}_{algorithm}_{formatted_date}_SEED{seed}.csv")

                # Precompute heatmaps and visited bins
                @dataclass
                class ProfileData:
                    profile_data: pd.DataFrame
                    visited_bins: any
                    heat_map: any

                func_timer = timer()  # Timer für die Precomputation starten
                print("Starting pre computation at: ", datetime.now())
                precomputed_profile_data_first = {
                    profile.iloc[0][identifier_column]: ProfileData(
                        profile_data=profile,
                        visited_bins=visited_bins(profile),
                        heat_map=create_heatmap(profile)
                    )
                    for profile in first_50
                }

                precomputed_profile_data_last = {
                    profile.iloc[0][identifier_column]: ProfileData(
                        profile_data=profile,
                        visited_bins=visited_bins(profile),
                        heat_map=create_heatmap(profile)
                    )
                    for profile in last_50
                }

                print("Pre computation took", timer() - func_timer, "seconds")

                # Iterate over all the baseline profiles (from first_50)
                j = 1
                for profile_id_baseline, data_baseline in precomputed_profile_data_first.items():
                    i = 1

                    smallest_distance = np.inf
                    smallest_distance_name = ""
                    obfuscation_factor = max(len(precomputed_profile_data_first[profile_id_baseline].profile_data) / 500, 50)
                    obfuscated_baseline, amount_iterations = heat_map_alteration(precomputed_profile_data_first[profile_id_baseline], obfuscation_factor, precomputed_profile_data_last, 1000)

                    for _, data_comparee in precomputed_profile_data_last.items():

                        topsoe_divergence = calculate_distance_between_points(obfuscated_baseline, data_comparee.heat_map)

                        if topsoe_divergence < smallest_distance and topsoe_divergence > 0:
                            smallest_distance = topsoe_divergence
                            smallest_distance_name = data_comparee.profile_data["FileName"].iloc[0]

                        if smallest_distance == 0:
                            raise ValueError("smallest_distance should never be = 0")

                        i += 1

                    results.append({
                        'Baseline': data_baseline.profile_data["FileName"].iloc[0],
                        'Comparee': smallest_distance_name,
                        'Distance': smallest_distance,
                        'Iterations': amount_iterations,
                        "Equal File Name": "Yes" if smallest_distance_name == data_baseline.profile_data["FileName"].iloc[0] else "No"
                    })

                    j += 1

                    if len(results) >= save_interval:
                        results_df = pd.DataFrame(results)
                        if os.path.exists(output_path):
                            results_df.to_csv(output_path, mode='a', header=False, index=False)
                        else:
                            results_df.to_csv(output_path, index=False)
                        results = []
                        gc.collect()

                # Save any remaining results
                if results:
                    results_df = pd.DataFrame(results)
                    if os.path.exists(output_path):
                        results_df.to_csv(output_path, mode='a', header=False, index=False)
                    else:
                        results_df.to_csv(output_path, index=False)
                    results = []
                    gc.collect()

                # Calculate additional metrics
                final_results_df = pd.read_csv(output_path)
                final_results_df['Row_Count'] = len(final_results_df)
                yes_count = final_results_df['Equal File Name'].value_counts().get('Yes', 0)
                no_count = final_results_df['Equal File Name'].value_counts().get('No', 0)
                total_count = yes_count + no_count
                final_results_df['Total_Yes'] = yes_count
                final_results_df['Total_No'] = no_count
                final_results_df['Percentage_Yes'] = (yes_count / total_count) * 100 if total_count > 0 else 0
                final_results_df['Percentage_No'] = (no_count / total_count) * 100 if total_count > 0 else 0

                final_results_df.to_csv(output_path, index=False)

    finally:
        stop_event.set()  # Set the event to stop the monitor thread

for seed in random_seeds:
    process_data(split_algorithm, scenarios, seed)

    
## Start the memory monitoring in a separate thread
#monitor_thread = threading.Thread(target=monitor_memory, args=(stop_event,))
#monitor_thread.daemon = True
#monitor_thread.start()

# Start the data processing in a separate thread
#processing_thread = threading.Thread(target=process_data, args=(split_algorithm, scenarios, stop_event))
#processing_thread.start()

# Wait for the processing to finish
#processing_thread.join()

# Ensure the monitor thread is stopped
#monitor_thread.join()

Synthetic data set selected
Starting pre computation at:  2024-07-20 12:31:25.001958
Pre computation took 31.768548699999883 seconds
Starting heat_map_alteration at: 2024-07-20 12:31:56.943162
Iteration 25418 finished. Topsoe Divergence H V = 434.82405926064627 Topsoe Divergence H U = 434.825021162104
heat_map_alteration execution finished after 19.902221799999097 seconds
Starting heat_map_alteration at: 2024-07-20 12:32:17.215544
Profile U: new_ifanfadd Profile H: new_abcoij belong to a different owner. No need for obfuscation. Returning H.
Starting heat_map_alteration at: 2024-07-20 12:32:17.682697
Profile U: new_ocshitcl Profile H: new_abdremlu belong to a different owner. No need for obfuscation. Returning H.
Starting heat_map_alteration at: 2024-07-20 12:32:18.153851
Iteration 1001 finished. Topsoe Divergence H V = 194.26148843877937 Topsoe Divergence H U = 42.121961100067566
heat_map_alteration execution finished after 1.0449277999996411 seconds
No solution found within 1001 iter

KeyboardInterrupt: 

In [36]:
### Scenario 2: Execute AP-Attack on the non obfuscated (NOBF) data sets  

"""
Select which data set shall be used
!! Important: The synthetic_profiles by default DO NOT have a Timestamp component, which means that splits BY DATE are NOT possible !!
"""
# Allowed keywords "SYNTHETIC" or "NORMAL"
scenarios = ["SYNTHETIC", "NORMAL"]
# Allowed keywords "HALF" or "RANDOM" or "CLUSTER"
split_algorithm = ["RANDOM"]

# Seeds generated with https://www.random.org/ True Random Number Generator, Params Min: 1, Max: 10000
random_seeds = [8046, 8251, 8166, 435, 7782, 9807, 3582, 21, 5446, 781]

def process_data(split_algorithm, scenarios, seed):
    for algorithm in split_algorithm:
        for scenario in scenarios:
            try:
                if scenario == "SYNTHETIC":
                    profiles_list_cleansed = remove_profiles_out_of_bounds_coordinates(synthetic_profiles)
                    print("Synthetic data set selected")
                elif scenario == "NORMAL":
                    profiles_list_cleansed = remove_profiles_out_of_bounds_coordinates(profiles_list)
                    print("Normal data set selected")
                else:
                    print(f"No dataset {scenario} could be found!")
                    sys.exit()

                # Select split function
                if algorithm == "HALF":
                    first_50, last_50 = split_data_frame_list_by_half(profiles_list_cleansed)
                elif algorithm == "RANDOM":
                    first_50, last_50 = split_data_frame_list_randomly(profiles_list_cleansed, seed)
                elif algorithm == "CLUSTER":
                    first_50, last_50 = split_data_frame_list_by_cluster(profiles_list)
                else:
                    print(f"No Split Algorithm {algorithm} could be found!")

                first_50 = list(filter(lambda df: not df.empty, first_50))
                last_50 = list(filter(lambda df: not df.empty, last_50))

                output_folder_path = Path('Daten/Ausgabe/AP Scenario NOB 50_50')
                output_folder = src_path / output_folder_path
                os.makedirs(output_folder, exist_ok=True)

                results = []

                i = 1
                j = 1
                for comparee_baseline in first_50:
                    i = 1

                    smallest_distance = np.inf
                    smallest_distance_name = ""
                    heatmap_baseline = create_heatmap(comparee_baseline)

                    for comparee_comparison in last_50:
                        heatmap_comparee = create_heatmap(comparee_comparison)

                        topsoe_divergence = calculate_distance_between_points(heatmap_baseline, heatmap_comparee)

                        if topsoe_divergence < smallest_distance:
                            smallest_distance = topsoe_divergence
                            smallest_distance_name = comparee_comparison["FileName"].iloc[0]

                        #print(f"Iteration: {j}.{i} Topsoe Divergence between {comparee_baseline['FileName'].iloc[0]} and {comparee_comparison['FileName'].iloc[0]}: {topsoe_divergence}")
                        i += 1

                    results.append({'Baseline': comparee_baseline["FileName"].iloc[0], 'Comparee': smallest_distance_name, 'Distance': smallest_distance, "Equal File Name": "Yes" if smallest_distance_name == comparee_baseline["FileName"].iloc[0] else "No"})
                    j += 1

                results_df = pd.DataFrame(results)
                results_df = results_df.sort_values("Distance")

                now = datetime.now()
                formatted_date = now.strftime("%Y%m%d_%H%M%S")

                output_path = os.path.join(output_folder, f"ap_output_NOB_{scenario}_{algorithm}_{formatted_date}_SEED{seed}.csv")
                results_df.to_csv(output_path, index=False)

                # Calculate additional metrics
                final_results_df = pd.read_csv(output_path)
                final_results_df['Row_Count'] = len(final_results_df)
                yes_count = final_results_df['Equal File Name'].value_counts().get('Yes', 0)
                no_count = final_results_df['Equal File Name'].value_counts().get('No', 0)
                total_count = yes_count + no_count
                final_results_df['Total_Yes'] = yes_count
                final_results_df['Total_No'] = no_count
                final_results_df['Percentage_Yes'] = (yes_count / total_count) * 100 if total_count > 0 else 0
                final_results_df['Percentage_No'] = (no_count / total_count) * 100 if total_count > 0 else 0

                final_results_df.to_csv(output_path, index=False)
            finally:
                gc.collect()

for seed in random_seeds:
    print(psutil.virtual_memory().percent)
    process_data(split_algorithm, scenarios, seed)

47.4
Synthetic data set selected
Normal data set selected
49.4
Synthetic data set selected
Normal data set selected
54.3
Synthetic data set selected


KeyboardInterrupt: 

In [ ]:
def fill_mobility_of_cell(list_of_all_available_time_gaps, set_of_mobility_traces_to_copy_from, max_speed_between_time_gaps, max_time_gap_inside_set_of_records, limit_duration_of_data_points_to_copy):
    
    """
    TODO: Complete function for mobility trace reconstruction. Check paper for rererence. 
    """
    # Filter list_of_all_available_time_gaps to fulfill being in cell i, j & are within max_speed_between_time_gaps

    # Split set_of_mobility_traces_to_copy_from by max_time_gap_inside_set_of_records

    # Select element from list_of_all_available_time_gaps & set_of_mobility_traces_to_copy_from with least distance. Distance is the sum of the distance\
    # from start of the gap to the set of records AND from the set of records to the ending point of the gap
    None

In [ ]:
def mobility_trace_reconstruction(obfuscated_heat_map, original_mobility_trace):

    """
    TODO: Complete function for mobility trace reconstruction. Check paper for rererence. 
    """

    modified_mobility_trace = modify_number_of_records(original_mobility_trace, len(obfuscated_heat_map))
    #modify_number_of_records(original_mobility_trace, amount_target_datapoints)
    #print(modified_mobility_trace)


profiles_list_cleansed = remove_profiles_out_of_bounds_coordinates(profiles_list)
#print(f"Amount of profiles before: {len(profiles_list)} Amount of profiles after: {len(profiles_list_cleansed)}")

first_80, last_20 = split_data_frame_list_by_half(profiles_list_cleansed)

first_80 = list(filter(lambda df: not df.empty, first_80))
last_20 = list(filter(lambda df: not df.empty, last_20))

profile_to_obfuscate = last_20[1]
#print(ele[:10])

obfuscated_heat_map = heat_map_alteration(profile_to_obfuscate, 100, first_80[:50], 1000) #heat_map_alteration(profiles_list[4], 0.5, profiles_list[5:100], len(profiles_list[4]), 10000)
    
results = mobility_trace_reconstruction(obfuscated_heat_map, profile_to_obfuscate)
#print(results)

